In [ ]:
# Libraries
import pandas as pd
import requests as rq
import re as re

In [ ]:
# Get all links of the indicators
df_links = pd.read_excel("Data-sheet-STUDENTS.xlsx", sheet_name="Data sources", header=None, usecols="C", skiprows=3, na_values="")
df_links = df_links.dropna()
links = []
for index, link in df_links[2].items():
    # We don't handle other source for now
    if "data.worldbank" not in link:
        continue
        
    # Match the useful indicator in the link
    match = re.search("indicator/([\.|\w]*)", link)
    links.append(match.group(1))

In [129]:
# Get countries data
df = pd.read_excel('Data-sheet-STUDENTS.xlsx', header=2, index_col=0)

In [ ]:
# Store the info of Country - ISO Code
iso_codes = {}

In [ ]:
# Get countries name for API request - Get ISO code
for index, name in df['Country (according to CN Gov):'].items():
    
    # Clean the data
    cleaned_name = name.split('(')[0].strip()
    if name == "Thailiand" :
        cleaned_name = "Thailand"
    if name == "The United Arab Emirates":
        cleaned_name = "United Arab Emirates"
    
    # We don't want to redo the request if the name already exists
    if name in iso_codes :
        continue
         
    # Request part
    apiURL = "https://restcountries.eu/rest/v2/name/" + cleaned_name
    resp = rq.get(apiURL)
        
    # Error handler
    if(resp.status_code != 200) :
        print("STATUS_CODE ERROR", resp.status_code, resp.text)
        continue
        
    # Response handler
    data = resp.json()
    alphaCode = data[0]['alpha3Code']
    if alphaCode == "IOT":
        alphaCode = "IND"
    iso_codes[name] = alphaCode

In [ ]:
# Create new column from response in the data frame
df['isocode'] = df['Country (according to CN Gov):'].apply(lambda x: iso_codes[x])

In [ ]:
# Store the info of ISO Code + the other indicators
countries_inds = {}

In [ ]:
# Get all columns name of the indicators
for indic in links:
    for country in iso_codes:

        # If the data is already retrieved we don't make the request
        if country not in countries_inds.keys():
            countries_inds[country] = {}
        elif indic in countries_inds[country].keys():
            continue
        
        # Request part
        apiURL = "http://api.worldbank.org/v2/country/"+ iso_codes[country] +"/indicator/"+ indic +"?format=json&mrv=1"
        resp = rq.get(apiURL)

        # Error handler
        if(resp.status_code != 200) :
            print("STATUS_CODE ERROR", resp.status_code, resp.text)
            print("Indicator : " + indic + " - country : " + iso_codes[country])
            print("URL : " + apiURL)
            continue

        # Response handler
        # print("indicator : " + indic + " - country : " + country)
        data = resp.json()
        if data[1] is None:
            indic_value = 0
        else :
            indic_value = data[1][0]['value']
        countries_inds[country].update({indic : indic_value})

In [ ]:
countries_inds

In [136]:
for country in countries_inds.keys():
    for indic in countries_inds[country]:
        if indic == "GC.TAX.IMPT.ZS":
            if pd.isnull( df['IMF WB\nCustoms and other import duties (% of tax revenue) 2016'].iloc[list(countries_inds).index(country)]):
                df['IMF WB\nCustoms and other import duties (% of tax revenue) 2016'].iloc[list(countries_inds).index(country)] = countries_inds[country][indic]
                # df.loc[list(countries_inds).index(country), 'IMF WB\nCustoms and other import duties (% of tax revenue) 2016'] = countries_inds[country][indic]
        elif indic == "FB.AST.NPER.ZS":
            if pd.isnull( df['WB IMF Non-performing loans to total gross loans ratio (%)'].iloc[list(countries_inds).index(country)]):
                df['WB IMF Non-performing loans to total gross loans ratio (%)'].iloc[list(countries_inds).index(country)] = countries_inds[country][indic]
        elif indic == "IS.RRS.GOOD.MT.K6":
            if pd.isnull( df['UIC Rail freight (mn ton x km travelled)'].iloc[list(countries_inds).index(country)]):
                df['UIC Rail freight (mn ton x km travelled)'].iloc[list(countries_inds).index(country)] = countries_inds[country][indic]
        elif indic == "IS.AIR.GOOD.MT.K1":
            if pd.isnull( df['ICAO air freight (mn ton x km travelled)'].iloc[list(countries_inds).index(country)]):
                df['ICAO air freight (mn ton x km travelled)'].iloc[list(countries_inds).index(country)] = countries_inds[country][indic]
        elif indic == "NE.CON.PRVT.PP.CD":
            if pd.isnull( df['WB Household final consumption expenditure, PPP bn USD'].iloc[list(countries_inds).index(country)]):
                df['WB Household final consumption expenditure, PPP bn USD'].iloc[list(countries_inds).index(country)] = countries_inds[country][indic]
        elif indic == "GB.XPD.RSDV.GD.ZS":
            if pd.isnull( df['WB UNESCO R&D Expenditure % of GDP'].iloc[list(countries_inds).index(country)]):
                df['WB UNESCO R&D Expenditure % of GDP'].iloc[list(countries_inds).index(country)] = countries_inds[country][indic]
        elif indic == "SP.POP.SCIE.RD.P6":
            if pd.isnull( df['WB UNESCO Researchers p. mn inhabitants'].iloc[list(countries_inds).index(country)]):
                df['WB UNESCO Researchers p. mn inhabitants'].iloc[list(countries_inds).index(country)] = countries_inds[country][indic]
        elif indic == "IC.REG.COST.PC.ZS":
            if pd.isnull( df['WB Cost of Starting a Business'].iloc[list(countries_inds).index(country)]):
                df['WB Cost of Starting a Business'].iloc[list(countries_inds).index(country)] = countries_inds[country][indic] 
        elif indic == "GC.DOD.TOTL.GD.ZS":
            if pd.isnull( df['WB Central Government Debt'].iloc[list(countries_inds).index(country)]):
                df['WB Central Government Debt'].iloc[list(countries_inds).index(country)] = countries_inds[country][indic] 
        elif indic == "SH.XPD.CHEX.PP.CD":
            if pd.isnull( df['WHO Current Health Expenditure p.c., PPP USD'].iloc[list(countries_inds).index(country)]):
                df['WHO Current Health Expenditure p.c., PPP USD'].iloc[list(countries_inds).index(country)] = countries_inds[country][indic]
        elif indic == "IP.PAT.RESD":
            if pd.isnull( df['WIPO Patent Applications'].iloc[list(countries_inds).index(country)]):
                df['WIPO Patent Applications'].iloc[list(countries_inds).index(country)] = countries_inds[country][indic] 
        elif indic == "IP.TMK.RSCT":
            if pd.isnull( df['WIPO Trademark applications'].iloc[list(countries_inds).index(country)]):
                df['WIPO Trademark applications'].iloc[list(countries_inds).index(country)] = countries_inds[country][indic]
        elif indic == "BX.KLT.DINV.WD.GD.ZS":
            if pd.isnull( df['IFDI Net Inflows (% of GDP)'].iloc[list(countries_inds).index(country)]):
                df['IFDI Net Inflows (% of GDP)'].iloc[list(countries_inds).index(country)] = countries_inds[country][indic]   


In [137]:
df

Country (according to CN Gov): Date of research inquiry  \
1                          Afghanistan                      NaT   
2                              Albania                      NaT   
3                              Armenia                      NaT   
4                           Azerbaijan                      NaT   
5                              Bahrain                      NaT   
6                           Bangladesh                      NaT   
7                              Belarus                      NaT   
8                               Bhutan                      NaT   
9               Bosnia and Herzegovina                      NaT   
10                              Brunei                      NaT   
11                            Bulgaria                      NaT   
12                            Cambodia               2018-08-16   
13                               China                      NaT   
14                             Croatia                      NaT   
15                      Czech Republic                      NaT   
16                          East Timor                      NaT   
17                               Egypt                      NaT   
18                             Estonia                      NaT   
19                    France (Not BRI)                      NaT   
20                             Georgia                      NaT   
21                   Germany (not BRI)                      NaT   
22                             Hungary                      NaT   
23                               India                      NaT   
24                           Indonesia                      NaT   
25                                Iran                      NaT   
26                                Iraq                      NaT   
27                              Israel                      NaT   
28                              Jordan                      NaT   
29                          Kazakhstan                      NaT   
30                              Kuwait                      NaT   
..                                 ...                      ...   
39                             Moldova                      NaT   
40                            Mongolia                      NaT   
41                          Montenegro                      NaT   
42                             Myanmar                      NaT   
43                               Nepal                      NaT   
44                                Oman                      NaT   
45                            Pakistan                      NaT   
46                         Philippines                      NaT   
47                              Poland                      NaT   
48                               Qatar                      NaT   
49                             Romania                      NaT   
50                              Russia                      NaT   
51                        Saudi Arabia                      NaT   
52                              Serbia                      NaT   
53                           Singapore                      NaT   
54                            Slovakia                      NaT   
55                            Slovenia                      NaT   
56                           Sri Lanka                      NaT   
57                               Syria                      NaT   
58                          Tajikistan                      NaT   
59                           Thailiand                      NaT   
60            The United Arab Emirates                      NaT   
61                              Turkey                      NaT   
62                        Turkmenistan                      NaT   
63            United Kingdom (Not BRI)                      NaT   
64  United States of America (Not BRI)                      NaT   
65                             Ukraine                      NaT   
66                          Uzbekistan                 